# Vector search

## Setup AI Search and OpenAI clients

In [12]:
import os

from azure.identity import AzureDeveloperCliCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from dotenv import load_dotenv
from openai import OpenAI

from render_table import render_product_results

load_dotenv()

azure_credential = AzureDeveloperCliCredential(tenant_id=os.environ["AZURE_TENANT_ID"])
token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)

openai_client = OpenAI(
    base_url=f"https://{os.environ['AZURE_OPENAI_SERVICE']}.openai.azure.com/openai/v1", api_key=token_provider
)

def get_embedding_vector(query: str) -> list[float]:
    response = openai_client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
        input=[query],
    )
    embedding_vector = response.data[0].embedding
    return embedding_vector

search_client = SearchClient(
    f"https://{os.environ['AZURE_SEARCH_SERVICE']}.search.windows.net",
    "zava-products-index",
    credential=azure_credential,
)

## Perform vector search

In [13]:
search_query = "water plants efficiently without waste"
search_vector = get_embedding_vector(search_query)

results = search_client.search(
    None, top=5, vector_queries=[VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding")]
)

render_product_results(results)

┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┓
┃ Sco… ┃ Name                 ┃ Description                                    ┃ Categories      ┃ Pri… ┃ SKU     ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━┩
│ 0.6… │ Self-Watering        │ Smart planter with built-in water reservoir    │ GARDEN &        │ $35… │ GOPLN0… │
│      │ Planter              │ for consistent plant hydration.                │ OUTDOOR,        │      │         │
│      │                      │                                                │ PLANTERS        │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.6… │ Soaker Hose 25-foot  │ Porous soaker hose for efficient water         │ GARDEN &        │ $25… │ GOHOS0… │
│      │                      │ delivery directly to plant roots.              │ OUTDOOR, HOSES  │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.5… │ Tomato Plant Food    │ Specialized fertilizer formulated for tomatoes │ GARDEN &        │ $14… │ GOFRT0… │
│      │                      │ and other fruiting plants.                     │ OUTDOOR,        │      │         │
│      │                      │                                                │ FERTILIZER      │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.5… │ Hanging Basket       │ Hanging basket with coco liner for flowers and │ GARDEN &        │ $15… │ GOPLN0… │
│      │ Planter              │ trailing plants.                               │ OUTDOOR,        │      │         │
│      │                      │                                                │ PLANTERS        │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.5… │ Plastic Planter Set  │ Lightweight plastic planters in multiple sizes │ GARDEN &        │ $21… │ GOPLN0… │
│      │                      │ with UV resistance.                            │ OUTDOOR,        │      │         │
│      │                      │                                                │ PLANTERS        │      │         │
└──────┴──────────────────────┴────────────────────────────────────────────────┴─────────────────┴──────┴─────────┘

## Perform vector search #2

In [11]:
search_query = "100 foot hose that wont break"
search_vector = get_embedding_vector(search_query)

results = search_client.search(
    None, top=5, vector_queries=[VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding")]
)

render_product_results(results)

┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┓
┃ Sco… ┃ Name                 ┃ Description                                    ┃ Categories      ┃ Pri… ┃ SKU     ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━┩
│ 0.7… │ Garden Hose 50-foot  │ Heavy-duty garden hose with brass fittings and │ GARDEN &        │ $32… │ GOHOS0… │
│      │                      │ kink-resistant construction.                   │ OUTDOOR, HOSES  │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.7… │ Heavy Duty Hose      │ Commercial-grade garden hose for heavy use     │ GARDEN &        │ $58… │ GOHOS0… │
│      │ 75-foot              │ with reinforced construction.                  │ OUTDOOR, HOSES  │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.7… │ Expandable Hose      │ Lightweight expandable hose that extends when  │ GARDEN &        │ $50… │ GOHOS0… │
│      │ 100-foot             │ water flows and contracts when emp...          │ OUTDOOR, HOSES  │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.6… │ Soaker Hose 25-foot  │ Porous soaker hose for efficient water         │ GARDEN &        │ $25… │ GOHOS0… │
│      │                      │ delivery directly to plant roots.              │ OUTDOOR, HOSES  │      │         │
├──────┼──────────────────────┼────────────────────────────────────────────────┼─────────────────┼──────┼─────────┤
│ 0.6… │ Drinking Water Safe  │ Lead-free garden hose safe for drinking water  │ GARDEN &        │ $40… │ GOHOS0… │
│      │ Hose                 │ and filling pools.                             │ OUTDOOR, HOSES  │      │         │
└──────┴──────────────────────┴────────────────────────────────────────────────┴─────────────────┴──────┴─────────┘